# 各種Seq2Seqモデルを試す

## simple_seq2seq

seq2seqの一番基本となる，4層のLSTMによるencoder-decoderモデル

### 準備

In [1]:
import pandas as pd
import numpy as np
import janome
from janome.tokenizer import Tokenizer


from collections import Counter
from keras.preprocessing import text, sequence
from keras.utils import np_utils

from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, CuDNNLSTM, TimeDistributed
from keras.optimizers import Adam

C:\Users\akio\Anaconda3\envs\keras-gpu\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
t = Tokenizer()
def get_wakati(text, eos=True, reverse=False):
    """
    text: 文書(str) 
    eos=Trueの場合は，end of sentence（テキストの終端）記号を付加する
    reverse=Trueの場合は，文字の並びを反転させる．（seq2seqでは，入力系列は反転しなければならない！）
    """
    wakati_list = t.tokenize(text, wakati=True)
    if reverse == True:
        wakati_list = list(reversed(wakati_list))
    if eos == True:
        wakati_list.append("<EOS>")
    return " ".join(wakati_list)

In [3]:
def words_count(df, target_colname):
    total_word_count = Counter()
    for text in df[target_colname]: 
        total_word_count += Counter(text.split())
    return len(total_word_count.keys())

In [4]:
def get_trn_feats(df, inp_colname="wakati_input", out_colname="wakati_output"):
    """
    分かち書き済みの文書(入出力系列)を，特徴エンジニアリングする．
    
    [arguments]
    df: 入出力関係を表すPandas DataFrame
    inp_colname: 入力（encoder）系列のカラム名
    out_colname: 出力（decoder）系列のカラム名
    
    [return values]
    token_input: 数値ベクトル化された入力系列
    onehot_output: one_hotベクトル化された出力系列
    NNモデルの出力層はデータ内の全単語に関するsoftmax値を求めるため，ラベルデータは予めonehot化する必要がある．
    tokenizer: 数値ベクトルに変換する際のモデル
    params: 入出力データに関するパラメータをdictとして纏めたもの
    """
    # 学習データの語彙数を計算する
    num_words = words_count(df, inp_colname) + words_count(df, out_colname)
    
    # テキストを数値ベクトル表現に変換する
    # 変換器の学習
    tokenizer = text.Tokenizer(num_words=num_words)
    inout_text_list = df[inp_colname].tolist() + df[out_colname].tolist()
    tokenizer.fit_on_texts(inout_text_list)
    # 最大の系列長を計算
    inout_text_len = [len(text) for text in inout_text_list]
    seq_maxlen = np.max(inout_text_len)
    # 数値ベクトルに変換
    token_input = tokenizer.texts_to_sequences(df[inp_colname].values)
    token_output = tokenizer.texts_to_sequences(df[out_colname].values)
    # 系列長を揃える
    token_input = sequence.pad_sequences(token_input, maxlen=seq_maxlen)
    token_output = sequence.pad_sequences(token_input, maxlen=seq_maxlen)
    # one-hot形式に変換
    onehot_output = np_utils.to_categorical(token_output, num_classes=num_words)
    
    # データに関する諸パラメータ
    params = {
        "seq_maxlen": seq_maxlen,
        "num_words": num_words
    }
    
    return token_input, onehot_output, tokenizer, params

In [5]:
def seq2seq_model(new_params=None):
    """
    最も基本的な4層LSTMによる，encoder-decoderモデル
    Ref: https://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf
    
    hyper_params: カスタム設定したハイパーパラメータ
    """
    params = {
        "seq_maxlen": 145,
        "num_words": 1616,
        "vec_len": 128,
        "learning_rate": 0.002,
        "dropout_rate": 0.5,
        "loss_func": "categorical_crossentropy",
        "metrics": ["accuracy"]
    }
    if new_params != None:
        params.update(new_params)
        
    inp = Input(shape=(params["seq_maxlen"], ))
    # end-to-end学習しやすいベクトルに写像するembedding層，その心はカーネルトリックに近い
    emb = Embedding(params["num_words"], params["vec_len"])(inp) 
    dout= SpatialDropout1D(params["dropout_rate"])(emb)
    # 論文同様に，4層LSTMを指定．return_sequences=Trueで時系列分のLSTM演算を実行する．
    lstmed = CuDNNLSTM(params["vec_len"], return_sequences=True)(dout)
    lstmed = CuDNNLSTM(params["vec_len"], return_sequences=True)(lstmed)
    lstmed = CuDNNLSTM(params["vec_len"], return_sequences=True)(lstmed)
    lstmed = CuDNNLSTM(params["vec_len"], return_sequences=True)(lstmed)
    out = TimeDistributed(Dense(params["num_words"], activation="softmax"))(lstmed)
    
    model = Model(inputs=inp, outputs=out)
    model.compile(loss=params["loss_func"], 
                  optimizer=Adam(lr=params["learning_rate"]),
                  metrics=params["metrics"]) # BLEUスコアで評価したいが，Kerasでは難しいので取り敢えず正解率で設定
    model.summary() # モデルの構成を表示
    return model

In [6]:
train_df = pd.read_csv("seq2seq_sample.csv")
train_df.head()

,__text,reaction
0,おはきゃーっと！,neglect
1,ねこますかわいいいいい,ありがと♡
2,のら！ちゃん！べりべりきゅーと！,neglect
3,おはきゃーっと！,neglect
4,🔥猫松🔥,猫松さんが燃やされていますね


In [7]:
train_df["wakati_input"] = train_df["__text"].apply(lambda x: get_wakati(x, reverse=True))
train_df["wakati_output"] = train_df["reaction"].apply(lambda x: get_wakati(x))
train_df.head()

,__text,reaction,wakati_input,wakati_output
0,おはきゃーっと！,neglect,！ っと きゃー は お <EOS>,neglect <EOS>
1,ねこますかわいいいいい,ありがと♡,いい いい わい か ます ねこ <EOS>,ありがと ♡ <EOS>
2,のら！ちゃん！べりべりきゅーと！,neglect,！ と ー ゅ き べり べり ！ ちゃん ！ のら <EOS>,neglect <EOS>
3,おはきゃーっと！,neglect,！ っと きゃー は お <EOS>,neglect <EOS>
4,🔥猫松🔥,猫松さんが燃やされていますね,🔥 松 猫 🔥 <EOS>,猫 松 さん が 燃やさ れ て い ます ね <EOS>


In [8]:
# 学習データの語彙数を計算する
nword_in = words_count(train_df, "wakati_input")
nword_out = words_count(train_df, "wakati_output")

In [9]:
# テキストを数値ベクトル表現に変換する
# 1. 変換器の学習
num_words = nword_in + nword_out
tokenizer = text.Tokenizer(num_words=num_words)
inout_text_list = train_df["wakati_input"].tolist() + train_df["wakati_output"].tolist()
tokenizer.fit_on_texts(inout_text_list)
# 2. 最大の系列長を計算
inout_text_len = [len(text) for text in inout_text_list]
seq_maxlen = np.max(inout_text_len)
# 3. 数値ベクトルに変換
token_input = tokenizer.texts_to_sequences(train_df["wakati_input"].values)
token_output = tokenizer.texts_to_sequences(train_df["wakati_output"].values)
# 系列長を揃える
token_input = sequence.pad_sequences(token_input, maxlen=seq_maxlen)
token_output = sequence.pad_sequences(token_output, maxlen=seq_maxlen)
# ラベルデータはsoftmaxで推定するので，予めone-hot形式に変換しておく必要がある．
onehot_output = np_utils.to_categorical(token_output, num_classes=num_words)

In [10]:
# 数値ベクトルに変換した特徴量の例
token_output[0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1])

In [11]:
onehot_output[0]

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]])

0は文字列なし，1はEOS．それ以外は，単語が数値に変換されている
逆変換する場合は，1以上の要素を抽出して，

```
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
```

を行えば良い？

In [12]:
# 数値データに戻す方法
np.argmax(onehot_output[0,143,:])

2

### 学習

#### Train-test split

In [13]:
seed = 7
val_ratio = 0.1
X_train, X_val, y_train, y_val = train_test_split(token_input, onehot_output, random_state=seed, test_size=val_ratio)

In [14]:
model = seq2seq_model()
# batch. epochsは論文を参考に設定．最適化はしていない
batch_size = 128
epochs = 8
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, 
                    validation_data=(X_val, y_val), 
                    # class_weight={0:0.01, 1:.99}, 
                    shuffle=True, verbose=1)

Instructions for updating:
Use the retry module or similar alternatives.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 145)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 145, 128)          206848    
_________________________________________________________________
spatial_dropout1d_1 (Spatial (None, 145, 128)          0         
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 145, 128)          132096    
_________________________________________________________________
cu_dnnlstm_2 (CuDNNLSTM)     (None, 145, 128)          132096    
_________________________________________________________________
cu_dnnlstm_3 (CuDNNLSTM)     (None, 145, 128)          132096    
_________________________________________________________________
cu_

### 推定

#### 準備

In [15]:
# 推定対称の入力系列を用意
example_testtext = "🔥猫松🔥" 
wakati_testtext= get_wakati(example_testtext, reverse=True)
# 数値ベクトルに変換　tokenizerは学習データの特徴量を作成したときのモデルを使用
token_testinput = tokenizer.texts_to_sequences(np.array([wakati_testtext]))
# 系列長を揃える
token_testinput = sequence.pad_sequences(token_testinput, maxlen=seq_maxlen)

In [16]:
onehot_test_output = model.predict(token_testinput, verbose=1)

1/1 [==============================] - 0s 326ms/step


In [17]:
# 推定したone-hotデータを数値ベクトルに変換する
token_testoutput = [np.argmax(seq_val) for seq_val in onehot_test_output[0]]

In [18]:
token_testoutput

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

推定したデータがすべて0になってしまう．考えられる原因：

- pad_sequenceで0で穴埋めしてしまったことにより，全ての値が0に過学習してしまった？
- 学習データ数が少なすぎる？

しかし，pad_sequenceで穴埋めしなければ，Kerasの学習は不可能？

解決案：
- 論文のように，系列長によってバッチを分ける？
- データ数を増やせば0に過学習しなくなる？